# **Redes Neuronales para PLN - Práctica Final**
## Notebook 5 - Resumen de Resultados y Conclusiones
### Alumno: Alvarez Lacasa, Lucas Alfredo

El objetivo de esta última notebook es el de reunir los resultados de cada una de las pruebas anteriores y mencionar algunas conclusiones, puntos de mejora y/o exploración de cada al futuro.

# Imports globales

Los imports son declarados en esta primera sección de la notebook

In [1]:

import os
from dataclasses import dataclass

# Funciones y clases auxiliares

Definiré un conjunto de funciones y clases utilitarias para evitar duplicación de código, haciendo que la notebook sea más mantenible y sencilla de volver a ejecutar. Creo que en general se trata de una buena práctica sobre todo en notebooks largas para poder aislar ejecución de código de forma de que sea menos probable cometer errores.

La gran mayoría de funciones y clases utilitarias (sin importar para qué sección del índice correspondan) serán listadas aquí por una cuestión de simplicidad y de poder re-cargar la notebook más fácilmente mientras desarrollo.

In [2]:
"""
=========================
Route resolution class

EDIT THIS CLASS depending on where you will be running this notebook!
=========================
"""

COLAB_BASE_MOUNT_POINT: str = "/content/drive"


@dataclass
class RouteConfig:
    notebook_name: str
    run_in_colab: bool = False
    # TODO: Change this to your base Colab path!!! =========>
    colab_base_mount_point: str = COLAB_BASE_MOUNT_POINT
    colab_practice_base_dir: str = f"{COLAB_BASE_MOUNT_POINT}/My Drive/UNED/master_nlp/RedesNeuronalesNLP/Practica_Final/challenge"
    local_practice_base_dir: str = "/home/lucas/Desktop/Personal/master_nlp/RedesNeuronalesNLP/Practica_Final/challenge"
    # ======================================================>
    dataset_rel_folder_path: str = os.path.join("data", "dataset", "OffendES")
    dataset_train_file_name: str = "train.csv"
    dataset_test_file_name: str = "test.csv"
    exp_output_rel_folder_path: str = "experiment_output"


class RouteResolver:
    """ This object abstracts the retrieval of the files used in this notebook by only editing the RouteConfig class received """

    def __init__(self, route_config: RouteConfig) -> None:
        if route_config.run_in_colab:
            # Import and mount into the base mount path for Google Colab
            from google.colab import drive
            drive.mount(route_config.colab_base_mount_point)

        self._config: RouteConfig = route_config

        # Create experiment output folder for this notebook
        exp_output_notebook_folder: str = self.current_exp_output_folder()
        if not os.path.isdir(exp_output_notebook_folder):
            print(f"Experiment output folder for this notebook does not exist. Creating...")
            os.mkdir(exp_output_notebook_folder)

    def base_folder_path(self) -> str:
        """Returns the base path depending if you're running in Colab or not"""
        return self._config.colab_practice_base_dir if self._config.run_in_colab else self._config.local_practice_base_dir

    def get_datasets_folder(self) -> str:
        """Returns the dataset folder"""
        return os.path.join(self.base_folder_path(), self._config.dataset_rel_folder_path)

    def get_training_dataset_path(self) -> str:
        """Returns path to the original training dataset file."""
        return os.path.join(self.get_datasets_folder(), self._config.dataset_train_file_name)

    def get_testing_dataset_path(self) -> str:
        """Returns path to the original test dataset file."""
        return os.path.join(self.get_datasets_folder(), self._config.dataset_test_file_name)

    def get_exp_output_folder(self) -> str:
        """Returns the path to the experiment outputs folder"""
        return os.path.join(self.base_folder_path(), self._config.exp_output_rel_folder_path)

    def get_exp_output_notebook_folder(self, flow_number: int) -> str:
        """Given flow number if returns the experiment output folder for it"""
        notebook_name: str = f"PracticaFinal_AlvarezLacasaLucas_{str(flow_number)}"
        exp_output_notebook_folder: str = os.path.join(self.get_exp_output_folder(), notebook_name)
        validate_condition(condition=os.path.isdir(exp_output_notebook_folder),
                           msg=f"Can't find experiment output for flow: {notebook_name}")
        return exp_output_notebook_folder

    def current_exp_output_folder(self) -> str:
        """Returns experiment output folder for current notebook being ran"""
        return os.path.join(self.get_exp_output_folder(), self._config.notebook_name)

    def dump_registered_paths(self) -> None:
        running_env: str = "COLAB" if self._config.run_in_colab else "LOCAL"
        print("Registered application paths ========")
        print(f"Running environment: {running_env}")
        print(f"Experiment output for this notebook is located in: {self.current_exp_output_folder()}")
        print(f"Original Training dataset file is located in: {self.get_training_dataset_path()}")
        print(f"Original Test dataset file is located in: {self.get_testing_dataset_path()}")

In [3]:
"""
=========================================
General utility functions
=========================================
"""


def validate_condition(condition: bool, msg: str, exc_type: type[Exception] = ValueError) -> None:
    """
    If condition is not met, it will raise a ValueError with the arguments provided
    :param condition: condition to validate
    :param msg: string to display in the error
    :param exc_type: type of exception we want to raise. ValueError by default
    :return: ValueError in case condition is not met, None otherwise
    """
    if not condition:
        raise exc_type(msg)


def create_dir_if_not_exists(folder: str) -> None:
    # Create a directory if it does not exist already
    if not os.path.isdir(folder):
        os.mkdir(folder)

# Resolución de rutas

Utilizaremos una clase `RouteResolver` para resolver las rutas, de forma de aislar esa información allí y poder modificarla en un único punto del notebook.

Verificar el output de la siguiente celda en caso de querer re-ejecutar la notebook para asegurarse de que las rutas apuntan al lugar correcto. Esta vez utilicé `run_in_colab=False` ya que todo el procesamiento pude hacerlo en mi portatil personal.

In [4]:
# Routes
route_resolver: RouteResolver = RouteResolver(
    RouteConfig(run_in_colab=False, notebook_name="5_results_and_conclusions"))
route_resolver.dump_registered_paths()

Experiment output folder for this notebook does not exist. Creating...
Registered application paths ========
Running environment: LOCAL
Experiment output for this notebook is located in: /home/lucas/Desktop/Personal/master_nlp/RedesNeuronalesNLP/Practica_Final/challenge/experiment_output/PracticaFinal_AlvarezLacasaLucas_5
Original Training dataset file is located in: /home/lucas/Desktop/Personal/master_nlp/RedesNeuronalesNLP/Practica_Final/challenge/data/dataset/OffendES/train.csv
Original Test dataset file is located in: /home/lucas/Desktop/Personal/master_nlp/RedesNeuronalesNLP/Practica_Final/challenge/data/dataset/OffendES/test.csv


# Resultados

El resumen de los mejores resultados de todas las notebooks anteriores se presenta en la siguiente tabla. _Notar que `MC` significa `MULTI-CLASS` y `BC` hace referencia a la performance para `BINARY-CLASS`_.. 

Para resolver este problema siempre se entrenaron los modelos sobre las 4 labels brindadas (`OFP`, `OFG`, `NO`, `NOM`). Para evaluar su desempeño y detectar `hate` o `no_hate`, las predicciones multi-class fueron mapeadas a su correspondiente binario siguiendo las reglas definidas en el enunciado.
En caso de haber entrenado algunos modelos únicamente usando las labels binarias, se podría quizás haber llegado a resultados superadores, puesto que la tarea que el clasificador tiene que resolver en ese caso sería mucho más sencilla que distinguir sobre las 4 labels como se está haciendo. Esto es un punto que podríamos considerar como de mejora.

| Notebook | Model                                                                                                                                      | Model Size | Dataset  | Representation                                                                    | macro-prec (MC) | macro-rec (MC) | macro-F1 (MC) | macro-prec (BC) | macro-rec (BC) | macro-F1 (BC) | Config highlights                                                                                                                                                                                                                                                                                                                                                                                     |
| -------- | ------------------------------------------------------------------------------------------------------------------------------------------ | ---------- | -------- | --------------------------------------------------------------------------------- |-----------------|-------------------|---------------|----------------------|-------------------|---------------| ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 2        | Naive Bayes                                                                                                                                |            | v2       | BOW: tf-idf<br>Level: word<br>N-grams: (1.3)                                      | 0.6000          | 0.5700            | 0.5706        | 0.7900               | 0.7500            | 0.7693        | \-                                                                                                                                                                                                                                                                                                                                                                                                    |
| 2        | LogisticRegression                                                                                                                         |            | v2       | BOW: tf-idf<br>Level: word<br>N-grams: (1.2)                                      | 0.6242          | 0.6001            | 0.6113        | 0.8045               | 0.7739            | 0.7880        | (C=10, class_weight='balanced', max_iter=500)                                                                                                                                                                                                                                                                                                                                                         |
| 2        | SVM                                                                                                                                        |            | v2       | BOW: tf-idf<br>Level: word<br>N-grams: (1.2)                                      | 0.6261          | 0.5951            | 0.6065        | 0.8149               | 0.7739            | 0.7923        | (C=1, class_weight='balanced', gamma=1, kernel='linear')                                                                                                                                                                                                                                                                                                                                              |
| 2        | RandomForests                                                                                                                              |            | v2       | BOW: tf-idf<br>Level: word<br>N-grams: (1.3)<br>Method: chi2 - 10k                | 0.5662          | 0.5229            | 0.5369        | 0.8153               | 0.6952            | 0.7350        | \-                                                                                                                                                                                                                                                                                                                                                                                                    |
| 2        | XGBoost                                                                                                                                    |            | v2       | BOW: tf-idf<br>Level: word<br>N-grams: (1.3)<br>Method: chi2 - 10k                | 0.5915          | 0.5579            | 0.5639        | 0.7961               | 0.7342            | 0.7597        | (learning_rate=0.1, max_depth=16, n_estimators=512)                                                                                                                                                                                                                                                                                                                                                   |
|          |                                                                                                                                            |            |          |                                                                                   |                 |                   |               |                      |                   |               |                                                                                                                                                                                                                                                                                                                                                                                                       |
| 3        | Dense NNs                                                                                                                                  | 4.27M      | v2       | Embeddings: from scratch<br>max_length=128<br>vocab_size=10001<br>output_dim: 100 | 0.6010          | 0.5275            | 0.5566        | 0.7871               | 0.7116            | 0.7407        | (batch_size=32, dense_dims=[256, 4], dense_act=["relu", "softmax"], dp_after_emb=0.5, dp_before_class_head=0.7, class_weight=None, Adam(learning_rate=0.001), EarlyStopping(monitor='val_loss', mode='min', patience=2, min_delta=0.01))                                                                                                                                                              |
| 3        | CNNs                                                                                                                                       | 1.2M       | v6       | Embeddings: from scratch<br>max_length=128<br>vocab_size=10001<br>output_dim: 100 | 0.5685          | 0.5548            | 0.5611        | 0.7238               | 0.7061            | 0.7144        | (batch_size=32, cnn_filter_dims: [64], cnn_kernel_sizes=[3], cnn_activations=["relu"], dense_act=["relu"], use_global_pooling=False, dp_after_emb=0.5, dp_after_pooling=[0.0], dp_before_class_head=0.1, dense_dims=[64, 4], dense_act=["relu", "softmax"], class_weight=None, Adam(learning_rate=0.001), EarlyStopping(monitor='val_loss', mode='min', patience=2, min_delta=0.01))                  |
| 3        | RNNs                                                                                                                                       | 1M         | v6       | Embeddings: from scratch<br>max_length=128<br>vocab_size=10001<br>output_dim: 100 | 0.6096          | 0.5736            | 0.5900        | 0.7761               | 0.7462            | 0.7598        | (batch_size=32, emb_dp=0.5, types=[RNNName.LSTM.value]\*2, units=[64, 32], bidirectional=True, unroll=True, internal_dp_rates=[0.0]\*2, recurrent_dp_rates=[0.1]\*2, external_dp_rates=[0.3]\*2, dense_dims=[128, 4], dense_dp=[0.3, 0.0], dense_act=["relu", "softmax"], class_weight=None, RMSprop(learning_rate=0.001), EarlyStopping(monitor='val_loss', mode='min', patience=2, min_delta=0.01)) |
|          |                                                                                                                                            |            |          |                                                                                   |                 |                   |               |                      |                   |               |                                                                                                                                                                                                                                                                                                                                                                                                       |
| 4        | [GPT 3.5 turbo](https://openai.com/blog/chatgpt) Zero Shot                                                                                 | 175B       | original | Tokenizer<br>Vocab: ~50k tokens<br>Max Input: 4096 tokens                         | 0.3811          | 0.5195            | 0.3560        | 0.6322               | 0.7451            | 0.7816        |                                                                                                                                                                                                                                                                                                                                                                                                       |
| 4        | [GPT-4](https://openai.com/research/gpt-4) Zero Shot                                                                                       | 1.75T      | v4       | Tokenizer<br>Vocab: ~100k tokens<br>Max Input: 8192 tokens                        | 0.4857          | 0.6398            | 0.5255        | 0.7099               | 0.8532            | 0.7419        |                                                                                                                                                                                                                                                                                                                                                                                                       |
| 4        | [mDeBERTa-v3-base-xnli-multilingual-nli-2mil7](https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7) Zero Shot | 279M       | v4       | Tokenizer<br>Vocab: ~ 30522 tokens<br>Max Input: 512 tokens                       | 0.3261          | 0.3511            | 0.2500        | 0.6355               | 0.7218            | 0.6522        |                                                                                                                                                                                                                                                                                                                                                                                                       |
| 4        | [robertuito-hate-speech](https://huggingface.co/pysentimiento/robertuito-hate-speech)                                                      | ~114M      | v4       | Tokenizer<br>Vocab: ~ 30k tokens<br>Max Input: 512 tokens                         | \-              | \-                | \-            | 0.6858               | 0.6584            | 0.6702        |                                                                                                                                                                                                                                                                                                                                                                                                       |
| 4        | [paraphrase-multilingual-MiniLM-L12-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) Setfit         | 118M       | v7       | Tokenizer<br>Vocab: ~ 250k tokens<br>Max Input: 128 tokens                        | 0.6361          | 0.6321            | 0.6339        | 0.7887               | 0.7795            | 0.7840        | (batch_size=16, num_iterations=3, num_epochs=1, add_early_stopping=False, early_stopping_patience=0)                                                                                                                                                                                                                                                                                                  |
| 4        | [distiluse-base-multilingual-cased-v1](https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v1) Setfit           | 135M       | v7       | Tokenizer<br>Vocab: ~ 120k tokens<br>Max Input: 128 tokens                        | 0.6251          | 0.6130            | 0.6189        | 0.8008               | 0.7901            | 0.7953        | (batch_size=16, num_iterations=2, num_epochs=1, add_early_stopping=False, early_stopping_patience=0)                                                                                                                                                                                                                                                                                                  |
| 4        | [LaBSE](https://huggingface.co/sentence-transformers/LaBSE) Setfit                                                                         | 471M       | v7       | Tokenizer<br>Vocab: ~ 501k tokens<br>Max Input: 256 tokens                        | 0.6941          | 0.6179            | **0.6495**    | 0.8257               | 0.7782            | **0.7992**    | (batch_size=16, num_iterations=2, num_epochs=3, add_early_stopping=False, early_stopping_patience=0)                                                                                                                                                                                                                                                                                                  |

_Para una mejor visualización, todos los resultados pueden visualizarse también en el siguiente [Google Spreadsheet](https://docs.google.com/spreadsheets/d/1mzY6YMFSP3kXgH3qgbJnX1w0f463ZQ06n32tLGgqOos/edit?usp=sharing)._



# Conclusiones

Las conclusiones presentadas en este caso serán más generales y abarcativas. La idea es brindar una imagen completa de lo que se probó y lo que no y comparar un poco la performance obtenida entre las distintas familias de modelos. Para las conclusiones más específicas de cada notebook, pueden ser encontradas al final de cada una de ellas.

**Highlights**:
- Se comenzó planteando una serie de baselines a través de modelos ligeros de Machine Learning. Inmediatamente el análisis de los datos sirvió para darnos cuenta del desbalance claro entre distintas clases. Se trabajó extensivamente con varias técnicas para mitigar este problema, incluyendo: Random Over Sampling, creación de ejemplos sintéticos con ChatGPT para la clase `OFG` y el uso de `class_weight="balanced"` en muchos modelos para poder modificar el peso que cada sample tiene en la función de pérdida en función de la clase a la que pertenece. En mayor o menor medida, todas estas técnicas contribuyeron notablemente a poder mejorar los resultados, ya que en las primeras pruebas eran realmente muy malos.
- Para la evaluación, se opta por reportar macro-averages, justamente porque el desbalance que hay entre la cantidad de samples de cada clase. De esta manera, al tener 4 labels, cada una de ellas aporta un 25% a los resultados. Este esquema es el recomendado para utilizar cuando se tienen grandes desbalances. Caso contrario, podríamos tener la sensación que el clasificador funciona muy bien si sólo predijera `NO`, cuando este no sería el caso.
- Se presentaron 8 versiones del dataset original, cada una con técnicas diferentes. Sin embargo, la cantidad de cleanups realizados fue baja. También se implementó una pipeline bastante genérica para poder reproducir este proceso de forma sencilla y confiable. Para modelos contextuales, se intento eliminar la información que realmente estuviéramos seguros de que podría introducir ruido, y se optó por dejar el resto para que se considere al momento de hacer el entrenamiento y la predicción. Esto sin duda fue un acierto.
- 🥇 **LaBSE SetFit**, con 64.95% de macro-f1 para MULTI-CLASS y 79.92% de macro-f1 BINARY-CLASS es el mejor modelo overall. Creo que esto era de esperarse, hablamos de un modelo de 471M de parámetros pre-entrenado en 109 idiomas y que luego fine-tuneamos para nuestra tarea en concreto. Tengo experiencia poniendo este modelo en concreto en producción y si bien no es extremadamente liviano, puede ejecutarse en CPU y lograr unas 5 inferencias por segundo aproximadamente.
- 🥈 Mención especial para `LogisticRegression`, que nos permite alcanzar un 61.13% de macro-f1 para MULTI-CLASS y 78.80% de macro-f1 para BINARY-CLASS. Probablemente parámetro-por-parámetro el mejor modelo de todos los que pudimos entrenar.
- 🥉 Un poco decepcionado por los modelos de Deep Learning que entrenamos (DNNs, CNNs y RNNs), los cuales tuvieron una performance muy parecida a los planteados como baseline basados en algoritmos de Machine Learning ligeros. Hubiese esperado poder alcanzar una mucha mejor performance con ellos (obviamente no tan buena como la que alcanzan los contextuales).
- A diferencia de la práctica anterior, la performance zero-shot de ChatGPT o GPT-4 es bastante pobre. Probablemente esto se deba a la dificultad de la tarea en este caso, a anotaciones que pueden ser un poco noisy y también al hecho de que están en español y utilizan un vocabulario muy particular.

_Para resumir, no me encuentro muy conforme con los resultados que alcancé. Siento que más allá de haber explorado todas las distintas arquitecturas y de haber gastado bastante tiempo itnentando mejorar los datos, siempre me encontré hiteando un ceiling que nunca pude sobrepasar. No estoy seguro si se debe a que hay anotaciones ruidosas (o inclusive que están mal) o a qué, pero es un aspecto interesante para seguir analizando. Me encantaría conocer cuál es la performance "state-of-the-art" esperada o alcanzada para este dataset._


**Trabajo futuro**:
- En cuanto a técnicas para combatir el desbalance del dataset, under-sampling no fue explorada. Se priorizó intentar mantener la mayor cantidad de información posible.
- Para modelos basados en Deep Learning (DNNs, CNNs, RNNs), no se exploraron representaciones de texto más simples, como BOW (TF, TF-IDF) ni selección de características sobre estas técnicas. Se priorizó ir directamente con un esquema mucho más clásico y eficiente basado en el uso de `Tokenizer` y `Embedding layer`.
- Se hicieron varios intentos por mejorar la performance obtenida a través de los datos. Sin embargo, fue común el caso para casi todos los modelos de tener un total de 20 elementos que solían predecirse como `NO` por nuestros modelos pero que estaban anotados como `OFG`. ¿Quien falla en esos casos? Deberíamos haber revisado a mano esos ejemplos para intentar descubrir si se trata de un error de anotación o si realmente los modelos están fallando y por qué. Haber mejorado sólo en esos pocos ejemplos nos hubiese permitido alcanzar una performance mucho más alta.
- No se hizo demasiada exploración sobre los optimizadores. Sacando el caso de las `RNNs` (donde se probó con `RMSprop`), siempre se usó Adam puesto que es un estándar en las pruebas.